## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,49.66,66,0,12.66,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,79.05,68,4,7.18,clear sky
2,2,Mataura,NZ,-46.1927,168.8643,59.49,37,10,3.56,clear sky
3,3,Vakhrushev,RU,48.9833,142.9667,16.61,76,100,6.93,overcast clouds
4,4,Carnarvon,AU,-24.8667,113.6333,100.47,13,11,14.97,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,79.05,68,4,7.18,clear sky
7,7,Mogadishu,SO,2.0371,45.3438,75.61,86,88,18.37,overcast clouds
12,12,Puerto Ayora,EC,-0.7393,-90.3518,75.54,92,75,4.00,broken clouds
14,14,Mahebourg,MU,-20.4081,57.7000,79.25,94,75,14.97,broken clouds
18,18,Mount Gambier,AU,-37.8333,140.7667,73.31,46,40,12.66,scattered clouds
19,19,Busselton,AU,-33.6500,115.3333,85.95,30,14,6.62,few clouds
22,22,Porto Belo,BR,-27.1578,-48.5531,78.78,90,100,8.10,overcast clouds
23,23,Saint-Philippe,RE,-21.3585,55.7679,81.99,89,100,25.64,moderate rain
28,28,Atuona,PF,-9.8000,-139.0333,78.37,77,25,15.90,scattered clouds
33,33,Cap Malheureux,MU,-19.9842,57.6142,79.47,82,99,10.00,overcast clouds


In [13]:
# 4a. Determine if there are any empty rows.
# check df for null values
preferred_cities_df.count()

City_ID                204
City                   204
Country                204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

preferred_cities_df = preferred_cities_df.dropna(how = 'any')
preferred_cities_df.count()

# No empty rows to drop. 

City_ID                204
City                   204
Country                204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,79.05,clear sky,-23.1203,-134.9692,
7,Mogadishu,SO,75.61,overcast clouds,2.0371,45.3438,
12,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,
14,Mahebourg,MU,79.25,broken clouds,-20.4081,57.7000,
18,Mount Gambier,AU,73.31,scattered clouds,-37.8333,140.7667,
19,Busselton,AU,85.95,few clouds,-33.6500,115.3333,
22,Porto Belo,BR,78.78,overcast clouds,-27.1578,-48.5531,
23,Saint-Philippe,RE,81.99,moderate rain,-21.3585,55.7679,
28,Atuona,PF,78.37,scattered clouds,-9.8000,-139.0333,
33,Cap Malheureux,MU,79.47,overcast clouds,-19.9842,57.6142,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df=hotel_df.dropna()
#hotel_df.head(30)
#hotel_df.isnull().sum

clean_hotel_df = hotel_df.dropna(how = 'any')
clean_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,79.05,clear sky,-23.1203,-134.9692,People ThankYou
7,Mogadishu,SO,75.61,overcast clouds,2.0371,45.3438,Hotel Juba
12,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Mahebourg,MU,79.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
18,Mount Gambier,AU,73.31,scattered clouds,-37.8333,140.7667,The Commodore
19,Busselton,AU,85.95,few clouds,-33.6500,115.3333,Observatory Guest House
22,Porto Belo,BR,78.78,overcast clouds,-27.1578,-48.5531,Costa Verde Beach Hotel
23,Saint-Philippe,RE,81.99,moderate rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
28,Atuona,PF,78.37,scattered clouds,-9.8000,-139.0333,Villa Enata
33,Cap Malheureux,MU,79.47,overcast clouds,-19.9842,57.6142,Pereybere Hotel & Spa


In [35]:
#Attempting to clear rows with no hotel name
clean2_hotel_df = clean_hotel_df.loc[clean_hotel_df['Hotel Name'] != '']
clean2_hotel_df.head(30)
#clean2_hotel.count()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,79.05,clear sky,-23.1203,-134.9692,People ThankYou
7,Mogadishu,SO,75.61,overcast clouds,2.0371,45.3438,Hotel Juba
12,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
14,Mahebourg,MU,79.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
18,Mount Gambier,AU,73.31,scattered clouds,-37.8333,140.7667,The Commodore
19,Busselton,AU,85.95,few clouds,-33.6500,115.3333,Observatory Guest House
22,Porto Belo,BR,78.78,overcast clouds,-27.1578,-48.5531,Costa Verde Beach Hotel
23,Saint-Philippe,RE,81.99,moderate rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
28,Atuona,PF,78.37,scattered clouds,-9.8000,-139.0333,Villa Enata
33,Cap Malheureux,MU,79.47,overcast clouds,-19.9842,57.6142,Pereybere Hotel & Spa


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean2_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean2_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean2_hotel_df[["Lat", "Lng"]]


In [56]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure


# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
#locations = hotel_df[["Lat", "Lng"]]
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))